In [1]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [2]:

# pip install mbutil

In [3]:
# lets generate the overview pictures 
#
floc = "tmp/county.geojson"
# gpd.read_file('data/CountiesUA.geojson')[['OBJECTID', 'CTYUA20CD', 'CTYUA20NM','LONG', 'LAT', 'geometry']].to_file(floc, driver="GeoJSON")

gdf = gpd.read_file('data/CountiesUA.geojson')[['CTYUA20CD', 'geometry']]
gdf.columns = ['id','geometry']
gdf.to_file(floc, driver="GeoJSON")


os.system('tippecanoe --read-parallel -Q -P -Z0 -z6 -f -o tiles/countries.mbtiles'+ floc)


tippecanoe: tiles/countries.mbtilestmp/county.geojson: unable to open database file


256

In [4]:
#  gpd.read_file('data/centroids2001.geojson').columns
gdf.head()


,id,geometry
0,E06000001,"POLYGON ((-1.27025 54.72717, -1.26822 54.72609..."
1,E06000002,"MULTIPOLYGON (((-1.19752 54.58211, -1.19434 54..."
2,E06000003,"MULTIPOLYGON (((-1.20098 54.57763, -1.20030 54..."
3,E06000004,"MULTIPOLYGON (((-1.27113 54.55427, -1.27213 54..."
4,E06000005,"POLYGON ((-1.63800 54.61720, -1.63767 54.61670..."


In [ ]:
year = '01'

base  = gpd.read_file('data/OA2001.geojson')[['OA01CD', 'geometry']]
base.columns = ['id','geometry']
centroids = gpd.read_file('data/centroids2001.geojson')[['OA01CD','geometry']]

print('save')
base.to_file("tmp/OA%s.geojson"%year, driver="GeoJSON")
centroids.to_file("tmp/centroids%s.geojson"%year, driver="GeoJSON")

print('process')
start = 6
end = 12
# cant have centroids being grouped, but fine for area shapes
os.system('tippecanoe --read-parallel -Q -Z%d -z%d -f --gamma=0 --no-feature-limit --no-tile-size-limit -r1 -o tiles/centroids%s.mbtiles  ./tmp/centroids%s.geojson'%(start,end,year,year))
os.system('tippecanoe --read-parallel -Q -Z%d -z%d -f -ab -o tiles/outputarea%s.mbtiles --drop-densest-as-needed --generate-id ./tmp/OA%s.geojson'%(start,end,year,year))



print('join')
os.system('tile-join -Q -P -o ./tiles/area_centroids_6_9.mbtiles -f ./tiles/centroids%s.mbtiles ./tiles/outputarea%s.mbtiles'%(year,year))

# --generate-id
# --coalesce-densest-as-needed

# tippecanoe -o tiles/test.mbtiles -Z9 -z10 -f ./data/tile_OA11.geojson ./data/tile_centroids11.geojson


# gdf_base = gdf_base.append(df,ignore_index=True)
# gdf_base .to_file("myfile.gpkg", layer="layer1", driver="GPKG")

save
process


For layer 0, using name "centroids01"
./tmp/centroids01.geojson:10944: Found ] at top level
./tmp/centroids01.geojson:10930: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
175434 features, 3073223 bytes of geometry, 16 bytes of separate metadata, 1929902 bytes of string pool

For layer 0, using name "OA01"
./tmp/OA01.geojson:23385: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
./tmp/OA01.geojson:9195: Found ] at top level
175434 features, 50496870 bytes of geometry, 895 bytes of separate metadata, 1929838 bytes of string pool
tile 6/32/21 size is 626726 with detail 12, >500000    
Going to try keeping the sparsest 71.80% of the features to make it fit
tile 6/32/21 size is 518544 with detail 12, >500000    
Going to try 

In [ ]:
# base  = gpd.read_file('data/OA2011.geojson')[['OA01CD', 'LAD01CD',
#        'geometry']]
# centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

# base.to_file("tmp/OA11.geojson", driver="GeoJSON")
# centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

# os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))




In [ ]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -Q -P -f ./tiles/countries.mbtiles ./tiles/area_centroids_6_9.mbtiles --attribution=ONS')

In [ ]:
# tippecanoe -z3 -o countries-z3.mbtiles --coalesce-densest-as-needed ne_10m_admin_0_countries.geojson
# tippecanoe -zg -Z4 -o states-Z4.mbtiles --coalesce-densest-as-needed --extend-zooms-if-still-dropping ne_10m_admin_1_states_provinces.geojson
# tile-join -o states-countries.mbtiles countries-z3.mbtiles states-Z4.mbtiles

In [ ]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf % 2>&1')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % mv % %.pbf 2>&1')


In [ ]:
os.system("git add -A; git commit -m 'recompute tiles'; git push")